# DT Assignment1

# Data Loading

In [1]:
import pandas as pd 
import numpy as np

In [2]:
pd_data = pd.read_csv('https://raw.githubusercontent.com/AugustLONG/ML01/master/01decisiontree/AllElectronics.csv')
pd_data.drop("RID",axis=1, inplace = True) #RID는 그냥 순서라서 삭제
pd_data

,age,income,student,credit_rating,class_buys_computer
0,youth,high,no,fair,no
1,youth,high,no,excellent,no
2,middle_aged,high,no,fair,yes
3,senior,medium,no,fair,yes
4,senior,low,yes,fair,yes
5,senior,low,yes,excellent,no
6,middle_aged,low,yes,excellent,yes
7,youth,medium,no,fair,no
8,youth,low,yes,fair,yes
9,senior,medium,yes,fair,yes


# 1. Gini 계수를 구하는 함수 만들기

- Input: df(데이터), label(타겟변수명)
- 해당 결과는 아래와 같이 나와야 합니다.

$ Gini(D_i) = 1 - \sum_j^m{p_j}^2$

In [11]:
# get_gini(pd_data,'class_buys_computer')

0.4591836734693877

In [4]:
def get_gini(df, label):
    
    # label의 class를 담은 list
    label_class = df[label].unique()
    
    # score : 각 class의 확률값의 제곱의 합
    score = 0
    for i in label_class:
        p = sum(df[label]==i) / len(df[label])
        score += p*p

    gini = 1 - score
    
    return gini    

In [5]:
get_gini(pd_data,'class_buys_computer')

0.4591836734693877

# 2. Feature의 Class를 이진 분류로 만들기
 ## ex) {A,B,C} -> ({A}, {B,C}), ({B}, {A,C}), ({C}, {A,B})

- Input: df(데이터), attribute(Gini index를 구하고자 하는 변수명)
- 해당 결과는 아래와 같이 나와야 합니다.

In [10]:
# get_binary_split(pd_data,"age")

[['youth'],
 ['middle_aged'],
 ['youth', 'middle_aged'],
 ['senior'],
 ['youth', 'senior'],
 ['middle_aged', 'senior']]

In [7]:
def get_binary_split(df, attribute):
    
    # gini index를 구하고자하는 변수의 class
    att_class = df[attribute].unique()
    result = []
    
    # 변수의 class가 3개 이상인 경우 차집합 개념을 통해 이진분류
    if len(att_class) > 2:
        for i in att_class:
            result.append([i])
            result.append(list(set(att_class)-set([i])))
    else:
        for i in att_class:
            result.append([i])
    
    return result

In [8]:
get_binary_split(pd_data,"age")

[['youth'],
 ['middle_aged', 'senior'],
 ['middle_aged'],
 ['youth', 'senior'],
 ['senior'],
 ['youth', 'middle_aged']]

# 3. 다음은 모든 이진분류의 경우의 Gini index를 구하는 함수 만들기
- 위에서 완성한 두 함수를 사용하여 만들어주세요!
- 해당 결과는 아래와 같이 나와야 합니다.

$Gini(A) = \sum_{j=1}^2 {|D_j|\over|D|}*Gini(D_i)$

In [10]:
# get_binary_split(pd_data,"age")

[['youth'],
 ['middle_aged'],
 ['youth', 'middle_aged'],
 ['senior'],
 ['youth', 'senior'],
 ['middle_aged', 'senior']]

- 특징: ex. age의 경우 gini_index 값을 비교해보면 이진분류된 class간의 gini index가 동일합니다. <br>
youth = middle_aged_senior <br>
middle_aged = youth_senior <br>
senior = youth_middle_aged <br>
<br>
(참고로, 식을 생각하면 당연한 이야기) <br>
$Gini_{age=youth} = (age=youth인\ Gini(D_i)) + (age=middle\ aged, senior인\ Gini(D_i))$

In [10]:
def get_attribute_gini_index(df, attribute, label):

    result = {}
    
    # 반복문을 통해 모든 이진분류의 경우의 gini index를 구합니다. 
    for i in get_binary_split(df, attribute):
        
        total = len(df[attribute]) # 해당 변수의 전체 갯수 (gini index 공식에서 D에 속함)
        count_1 = count_2 = 0 # 해당 변수의 class에 해당하는 갯수 초기화 (gini index 공식에서 Dj에 속함)
        
        # 이진분류의 경우에 두 개 이상의 원소를 갖는 경우가 있으므로 케이스를 분리해서 gini index를 구합니다. 
        # 두 개 이상의 원소를 갖는 경우
        if len(i)>1:
            # 위에서 언급한 특징에서 idea를 얻어, 두 개 이상의 원소에 해당하지 않는, 나머지 원소의 gini 계수를 구해줍니다. 
            gini_1 = get_gini(df[df[attribute]==''.join(set(df[attribute].unique())-set(i))], 'class_buys_computer')
            gini_2 = get_gini(df[df[attribute]!=''.join(set(df[attribute].unique())-set(i))], 'class_buys_computer')

            count_1 = sum(df[attribute]==''.join(set(df[attribute].unique())-set(i)))
            count_2 = total - count_1

        # 한 개의 원소를 갖는 경우
        else:
            gini_1 = get_gini(df[df[attribute]==''.join(i)], 'class_buys_computer')
            gini_2 = get_gini(df[df[attribute]!=''.join(i)], 'class_buys_computer')

            count_1 = sum(df[attribute]==''.join(i))
            count_2 = total - count_1

        # gini index를 구해줍니다. 
        gini_index = (count_1/total)*gini_1 + (count_2/total)*gini_2

        result['_'.join(i)] = gini_index
    
    return result

In [11]:
get_attribute_gini_index(pd_data, "age", "class_buys_computer")

{'youth': 0.3936507936507937,
 'middle_aged_senior': 0.3936507936507937,
 'middle_aged': 0.35714285714285715,
 'youth_senior': 0.35714285714285715,
 'senior': 0.45714285714285713,
 'youth_middle_aged': 0.45714285714285713}

여기서 가장 작은 Gini index값을 가지는 class를 기준으로 split해야겠죠?

In [12]:
min(get_attribute_gini_index(pd_data, "age", "class_buys_computer").items())

('middle_aged', 0.35714285714285715)

# 다음의 문제를 위에서 작성한 함수를 통해 구한 값으로 보여주세요!
## 문제1) 변수 ‘income’의 이진분류 결과를 보여주세요.

## 문제2) 분류를 하는 데 가장 중요한 변수를 선정하고, 해당 변수의 Gini index를 제시해주세요.

## 문제3) 문제 2에서 제시한 feature로 DataFrame을 split한 후 나눠진 2개의 DataFrame에서 각각   다음으로 중요한 변수를 선정하고 해당 변수의 Gini index를 제시해주세요.

In [13]:
##문제1 답안

get_binary_split(pd_data, 'income')

[['high'],
 ['low', 'medium'],
 ['medium'],
 ['low', 'high'],
 ['low'],
 ['high', 'medium']]

In [14]:
##문제2 답안

def important_feature(df, label):
    # 각 feature의 minimum gini index와 class를 구합니다. 
    minimum = {}

    for i in df.columns[:-1]:
        minimum[i] = min(get_attribute_gini_index(df, i, label).items())

    # 반복문을 돌면서 가장 작은 불순도를 갖는 변수와 해당 변수의 gini index를 구합니다. 
    result = ''
    c = 1
    
    for k, v in minimum.items():
        if v[1] < c :
            result = str(k)+':'+str(v)
            c = v[1]
        
        # gini index가 동일하다면, 동일한 모든 변수 및 gini index를 출력합니다. 
        elif v[1] == c :
            result += ('\n'+str(k)+':'+str(v))
 
    return result

important_feature(pd_data, 'class_buys_computer')

"age:('middle_aged', 0.35714285714285715)"

## 문제3) 문제 2에서 제시한 feature로 DataFrame을 split한 후 나눠진 2개의 DataFrame에서 각각   다음으로 중요한 변수를 선정하고 해당 변수의 Gini index를 제시해주세요.

In [17]:
##문제3 답안

# age가 middle_aged인 dataframe과 youth_senior인 dataframe으로 구분합니다. 
df1 = pd_data.query("age=='middle_aged'")
df2 = pd_data.query("age=='youth'| age=='senior'")

# df1에서의 중요한 변수 선정 및 gini index를 구합니다. 
print('--- df1에서의 주요 변수 및 Gini index ---')
print(important_feature(df1, 'class_buys_computer'))

print('\n')

# df2에서의 중요한 변수 선정 및 gini index를 구합니다. 
print('--- df2에서의 주요 변수 및 Gini index ---')
print(important_feature(df2, 'class_buys_computer'))

--- df1에서의 주요 변수 및 Gini index ---
age:('middle_aged', 0.0)
income:('high', 0.0)
student:('no', 0.0)
credit_rating:('excellent', 0.0)


--- df2에서의 주요 변수 및 Gini index ---
student:('no', 0.31999999999999984)
